In [4]:
import pandas as pd
import numpy as np
df = pd.read_csv('train.csv')

In [ ]:
df = df.reset_index()
df=df.rename(columns = {'index':'ImageId'})

In [5]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.171-b11, mixed mode)
  Starting server from c:\users\drsandeepabrol\appdata\local\programs\python\python36\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\DRSAND~1\AppData\Local\Temp\tmp289w9rh8
  JVM stdout: C:\Users\DRSAND~1\AppData\Local\Temp\tmp289w9rh8\h2o_drsandeepabrol_started_from_python.out
  JVM stderr: C:\Users\DRSAND~1\AppData\Local\Temp\tmp289w9rh8\h2o_drsandeepabrol_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,03 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.11
H2O cluster version age:,1 month and 10 days
H2O cluster name:,H2O_from_python_drsandeepabrol_rbydpr
H2O cluster total nodes:,1
H2O cluster free memory:,1.757 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [23]:
hf=h2o.H2OFrame(df)

c:\users\drsandeepabrol\appdata\local\programs\python\python36\lib\site-packages\h2o\utils\shared_utils.py:170: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  data = _handle_python_lists(python_obj.as_matrix().tolist(), -1)[1]


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [22]:
hf.columns

['label',
 'pixel0',
 'pixel1',
 'pixel2',
 'pixel3',
 'pixel4',
 'pixel5',
 'pixel6',
 'pixel7',
 'pixel8',
 'pixel9',
 'pixel10',
 'pixel11',
 'pixel12',
 'pixel13',
 'pixel14',
 'pixel15',
 'pixel16',
 'pixel17',
 'pixel18',
 'pixel19',
 'pixel20',
 'pixel21',
 'pixel22',
 'pixel23',
 'pixel24',
 'pixel25',
 'pixel26',
 'pixel27',
 'pixel28',
 'pixel29',
 'pixel30',
 'pixel31',
 'pixel32',
 'pixel33',
 'pixel34',
 'pixel35',
 'pixel36',
 'pixel37',
 'pixel38',
 'pixel39',
 'pixel40',
 'pixel41',
 'pixel42',
 'pixel43',
 'pixel44',
 'pixel45',
 'pixel46',
 'pixel47',
 'pixel48',
 'pixel49',
 'pixel50',
 'pixel51',
 'pixel52',
 'pixel53',
 'pixel54',
 'pixel55',
 'pixel56',
 'pixel57',
 'pixel58',
 'pixel59',
 'pixel60',
 'pixel61',
 'pixel62',
 'pixel63',
 'pixel64',
 'pixel65',
 'pixel66',
 'pixel67',
 'pixel68',
 'pixel69',
 'pixel70',
 'pixel71',
 'pixel72',
 'pixel73',
 'pixel74',
 'pixel75',
 'pixel76',
 'pixel77',
 'pixel78',
 'pixel79',
 'pixel80',
 'pixel81',
 'pixel82',
 'pi

In [25]:
from h2o.automl import H2OAutoML
x=hf.columns
y='label'
hf['label']=hf['label'].asfactor()
for r in ['ImageId',y]:
    x.remove(r)
    
aml = H2OAutoML(max_runtime_secs = 60)
aml.train(x = x, y = y,
          training_frame = hf)

AutoML progress: |████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [26]:
aml.leaderboard

model_id,mean_per_class_error
DRF_0_AutoML_20180704_184120,0.0521933


In [ ]:
df.columns

In [27]:
test=h2o.H2OFrame(pd.read_csv('test.csv'))
preds=aml.predict(test)
final=test.cbind(preds)
results=h2o.as_list(final)

c:\users\drsandeepabrol\appdata\local\programs\python\python36\lib\site-packages\h2o\utils\shared_utils.py:170: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  data = _handle_python_lists(python_obj.as_matrix().tolist(), -1)[1]


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%


In [29]:
odf=results.loc[:,['ImageId','predict']]
odf.columns=['ImageId','label']
odf.to_csv('submission.csv',index=False)

c:\users\drsandeepabrol\appdata\local\programs\python\python36\lib\site-packages\pandas\core\indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [31]:
df2 = pd.read_csv('submission.csv')
df3 = pd.read_csv('sample_submission.csv')
df2.ImageId = df3.ImageId

In [35]:
df3.columns

Index(['ImageId', 'Label'], dtype='object')

In [40]:
df2.to_csv('submission2.csv',index=False)